#Installing Dependencies

In [ ]:
# !pip install openai datasets
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 KB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.0-py3-none-any.whl size=66855 sha256=7043561d7079be8244458bd85f90f2623a5bc3ad339e359719c7e63a87d2f854
  Stored in directory: /root/.cache/pip/wheels/7e/4c/c8/31e9d441bd937e2b9076627465f9db43ab6db40f08aae60b66
Successfully built openai


In [ ]:
!pip install -U nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing Packages

In [ ]:
import ast 
import os
# import openai
import pandas as pd

# from datasets import load_metric

# Loading Data

In [ ]:
difficult_ncs_df_raw = pd.read_csv('export_set_1.csv')
diff_dict = {'nc':[]}

for index, row in difficult_ncs_df_raw.iterrows():
  nc = row['nc']
  diff_dict['nc'].append(nc)

diff_df = pd.DataFrame.from_dict(diff_dict)

In [ ]:
train_df_raw = pd.read_csv('train_gold.csv')
train_dict = {'nc':[], 'paraphrase':[]}

for index, row in train_df_raw.iterrows():
  nc = row['w1'] + " " + row['w2']
  train_dict['nc'].append(nc)
  train_dict['paraphrase'].append(row['paraphrase'])

train_df = pd.DataFrame.from_dict(train_dict)
train_df

,nc,paraphrase
0,advertising agency,agency that does advertising
1,advertising agency,agency that does advertising business
2,advertising agency,agency that does work in advertising
3,advertising agency,agency that does work in the advertising field
4,advertising agency,agency that does work in the advertising business
...,...,...
5436,work area,area in which one works
5437,work area,area in which one is working
5438,work area,area in which work occurs
5439,work area,area in which work happens


In [ ]:
def load_test_valid_df(file_path):
    df = pd.read_csv(file_path, usecols=['nc','paraphrases'])

    for i, row in df.iterrows():
        string_literal = df.at[i, 'paraphrases']
        df.at[i, 'paraphrases'] = ast.literal_eval(string_literal)

    return df

test_df = load_test_valid_df('test_df.csv')
print(test_df)

                       nc                                        paraphrases
0            police court  [court for minor offenders arrested by the pol...
1           business unit  [unit of business, unit of a business, unit of...
2          signature tune  [tune that someone makes their signature, tune...
3         spring sunshine  [sunshine of spring, sunshine of the spring, s...
4           reflex action  [action caused by reflex, action caused by a r...
..                    ...                                                ...
105     middle management  [management at middle level, management at the...
106          woman worker  [woman who works, woman who works a job, woman...
107  virtuoso performance  [performance by a virtuoso, performance done b...
108  union representative  [representative of a union, representative of ...
109        pension scheme  [scheme defining a pension, scheme defining a ...

[110 rows x 2 columns]


In [ ]:
valid_df = load_test_valid_df('valid_df.csv')
print(valid_df)

                            nc  \
0             daytime activity   
1               welfare system   
2              washing machine   
3                  mining area   
4              state socialism   
5               amateur record   
6                   ruler line   
7                 sport editor   
8                   air filter   
9                   love scene   
10     interrogation technique   
11           publicity purpose   
12              property price   
13                  drug baron   
14              summer meeting   
15                  place card   
16             transfer market   
17              village school   
18        fertility counsellor   
19             coalition force   
20               gene mutation   
21            transaction cost   
22                cabinet room   
23                 home market   
24                  water bath   
25                 drawing pin   
26  telecommunications licence   
27           election campaign   

             

In [ ]:
countparas = 0
for i, row in test_df.iterrows():
  countparas += len(row['paraphrases'])

print(countparas)

4820


In [ ]:
countparas = 0
for i, row in valid_df.iterrows():
  countparas += len(row['paraphrases'])

print(countparas)

1469


In [ ]:
import pandas as pd

test_df = pd.read_csv("new_diff_set.csv", usecols=['nc'])
test_df

,nc
0,accounting sleight
1,alchemy observation
2,art governor
3,assault prosecution
4,atomization result
...,...
100,voice shake
101,wack philosophy
102,warrior gang
103,weiner race


In [ ]:
def get_random_prompt(train_df, test_df, num_egs=1):
  test_row = test_df.sample()
  test_nc = test_row['nc'].values[0]
  test_paras = test_row['paraphrases'].values[0]

  prompt = ""
  
  prompt += train_nc_prompt(train_df, num_egs)
  prompt += test_nc_prompt(test_nc)
  
  return prompt, test_nc, test_paras

def train_nc_prompt(train_df, num_egs, prompt_text="Q: what is the meaning of {}?\n"):
  prompt = ""
  for i in range(num_egs):
    train_row = train_df.sample()
    train_nc = train_row['nc'].values[0]
    train_para = train_row['paraphrase'].values[0]

    train_q = prompt_text.format(train_nc)
    train_a = "A: " + train_para + "\n"

    prompt += train_q + train_a
  return prompt

def test_nc_prompt(test_nc, prompt_text="Q: what is the meaning of {}?\n"):
  test_q = prompt_text.format(test_nc)
  test_a = "A:"
  return test_q + test_a

def get_all_test_prompts(train_df, test_df, num_egs=10, prompt_text="Q: what is the meaning of {}?\n"):
  prompts = []
  for i, test_row in test_df.iterrows():
    test_nc = test_row['nc']
    gold_paras = test_row['paraphrases']

    prompt = ""
    prompt += train_nc_prompt(train_df, num_egs, prompt_text)
    prompt += test_nc_prompt(test_nc, prompt_text)

    prompts.append((test_nc, gold_paras, prompt))
  
  return prompts

def get_all_diff_prompts(train_df, diff_df, num_egs=10, prompt_text="Q: what is the meaning of {}?\n"):
  prompts = []
  for i, row in diff_df.iterrows():
    diff_nc = row['nc']

    prompt = ""
    prompt += train_nc_prompt(train_df, num_egs, prompt_text)
    prompt += test_nc_prompt(diff_nc, prompt_text)

    prompts.append((diff_nc, prompt))
  
  return prompts

In [ ]:
prompt = train_nc_prompt(train_df, 10) + test_nc_prompt("graduation parking")
print(prompt)

Q: what is the meaning of history book?
A: book with history
Q: what is the meaning of household account?
A: investment account for an entire household
Q: what is the meaning of police spokesman?
A: spokesman who speaks on the behalf of the police
Q: what is the meaning of bank deposit?
A: deposit at a bank
Q: what is the meaning of corporation tax?
A: tax that corporations are required to pay
Q: what is the meaning of divorce rate?
A: rate of divorce
Q: what is the meaning of telephone conversation?
A: conversation held through a telephone
Q: what is the meaning of census record?
A: record of a census
Q: what is the meaning of music television?
A: television channel where music is shown
Q: what is the meaning of fire officer?
A: officer who fights fire
Q: what is the meaning of graduation parking?
A:


In [ ]:
openai.organization = "<ORGANIZATION-KEY>"
openai.api_key = "<API-KEY>"

In [ ]:
# difficult NC generation
import time

num_egs = 1
all_prompts = get_all_diff_prompts(train_df, test_df, num_egs, "Q: what is the meaning of {}?\n")

text_responses = {}
curr_index = 0

for diff_nc, prompt in all_prompts[curr_index:]:
  # generate number of responses equal to the number of gold paraphrases
  time.sleep(15)
  num_to_gen = 5
  generated_paras = []
  responses = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt,
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n=num_to_gen
  )
  for r in responses['choices']:
    generated_paras.append(r.text.lstrip())

  curr_index += 1
  print("'{}': '{}'".format(diff_nc, generated_paras[0]))
  
  text_responses[diff_nc] = generated_paras

In [ ]:
import time

num_egs = 1
all_prompts = get_all_test_prompts(train_df, test_df, num_egs, "Q: what is the meaning of {}?\n")

text_responses = {}
curr_index = 0

# for test_nc, gold_paras, prompt in all_prompts:
for test_nc, gold_paras, prompt in all_prompts[curr_index:]:
  # generate number of responses equal to the number of gold paraphrases
  time.sleep(15)
  num_to_gen = min(len(gold_paras),128)
  generated_paras = []
  responses = openai.Completion.create(
    model="text-davinci-002",
    prompt=prompt,
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    n=num_to_gen
  )
  for r in responses['choices']:
    generated_paras.append(r.text.lstrip())

  curr_index += 1
  print("'{}': '{}'".format(test_nc, generated_paras[0]))
  
  text_responses[test_nc] = generated_paras

In [ ]:
mydict = {'nc': [], 'paraphrase': []}

for nc, paras in text_responses.items():
  for p in paras:
    mydict['nc'].append(nc)
    mydict['paraphrase'].append(p)

df = pd.DataFrame(mydict)
df

,nc,paraphrase
0,accounting sleight,a clever or dishonest act or scheme in accounting
1,accounting sleight,an accounting method or technique that is inte...
2,accounting sleight,"a skillful maneuver, especially in financial d..."
3,accounting sleight,accounting deception
4,accounting sleight,A dishonesty in accounting in order to make fi...
...,...,...
520,wheel efficiency,the efficiency of a wheel in turning or transl...
521,wheel efficiency,how much work a wheel can do in relation to th...
522,wheel efficiency,The efficiency of a wheel is the Wheel's outpu...
523,wheel efficiency,the percentage of power from the engine that i...


In [ ]:
df.to_csv('gpt3-responses_new_diff_set.csv')